## Quant Research:

* Analysing the distribution of a stock returns
* Deriving a stock beta
* Determining correlations between two or more stocks
* Calculating rate of change
* Backtesting a strategy

While choosing portfolio, choose uncorelated stocks

Decide your strategy as per the momentum of the market.


### Backtest Moving Average Crossover on multiple Stocks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# matplotlib inline

In [ ]:
import yfinance as yf

# define stock list

stock_list = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG']

#DEFINE STRATEGY PARAMS
SMA = 10
LMA =